In [1]:
import numpy as np
from collections import defaultdict

class MarkovChainMLE:
    def __init__(self):
        self.transition_matrix = None
        self.states = None
        self.state_to_idx = None
        
    def fit(self, sequence):
        # First, identify all unique states
        unique_states = sorted(set(sequence))
        self.states = unique_states
        self.state_to_idx = {state: idx for idx, state in enumerate(unique_states)}
        n_states = len(unique_states)
        
        # Initialize count matrix
        counts = np.zeros((n_states, n_states))
        
        # Count transitions
        for t in range(len(sequence)-1):
            current_state = sequence[t]
            next_state = sequence[t+1]
            i = self.state_to_idx[current_state]
            j = self.state_to_idx[next_state]
            counts[i, j] += 1
            
        # Convert counts to probabilities using MLE
        # MLE estimate = (number of transitions from i to j) / (total transitions from i)
        row_sums = counts.sum(axis=1, keepdims=True)
        # Avoid division by zero
        row_sums[row_sums == 0] = 1
        self.transition_matrix = counts / row_sums
        
    def predict_next_state_proba(self, current_state):
        """Predict probability distribution of next state"""
        state_idx = self.state_to_idx[current_state]
        return self.transition_matrix[state_idx]
    
    def generate_sequence(self, start_state, length):
        """Generate a sequence of states"""
        sequence = [start_state]
        current_state = start_state
        
        for _ in range(length-1):
            # Get probabilities for next state
            probs = self.predict_next_state_proba(current_state)
            # Sample next state
            next_state = np.random.choice(self.states, p=probs)
            sequence.append(next_state)
            current_state = next_state
            
        return sequence

# Let's demonstrate with a weather example
def demonstrate_weather_markov():
    # Example sequence: Sunny (S), Rainy (R), Cloudy (C)
    weather_sequence = ['S', 'S', 'R', 'R', 'C', 'S', 'R', 'C', 'S', 'S']
    
    # Fit Markov chain
    mc = MarkovChainMLE()
    mc.fit(weather_sequence)
    
    print("Transition Matrix:")
    for i, state in enumerate(mc.states):
        print(f"\nFrom {state} to:")
        for j, next_state in enumerate(mc.states):
            prob = mc.transition_matrix[i, j]
            print(f"{next_state}: {prob:.2f}", end=' ')
            
    # Generate a new sequence
    print("\n\nGenerated weather sequence:")
    new_sequence = mc.generate_sequence('S', 5)
    print(' -> '.join(new_sequence))
    
    return mc

In [2]:
demonstrate_weather_markov()

Transition Matrix:

From C to:
C: 0.00 R: 0.00 S: 1.00 
From R to:
C: 0.67 R: 0.33 S: 0.00 
From S to:
C: 0.00 R: 0.50 S: 0.50 

Generated weather sequence:
S -> S -> S -> R -> C
